# Convert the Radar Data Format to Row-Wise Format

In [1]:
import pandas as pd

## Read in the data

In [2]:
df = pd.read_csv("rw_east_thru_summary.csv")
df = df.loc[df['r2'] > 0.9].copy()
df = df.loc[df.type.isin(['decel', 'accel'])].copy()

In [3]:
types_df = pd.get_dummies(df[['vehicle_id', 'type']]).groupby('vehicle_id').sum()

In [4]:
types_df.loc[(types_df['type_accel'] > 0) & (types_df['type_decel'] > 0)].shape

(1173, 2)

In [5]:
types_df.loc[(types_df['type_accel'] > 0), 'type_accel'].sum(), types_df.loc[(types_df['type_accel'] > 0), 'type_accel'].shape 

(3918, (2770,))

In [6]:
types_df.loc[(types_df['type_decel'] > 0), 'type_decel'].sum(), types_df.loc[(types_df['type_decel'] > 0), 'type_decel'].shape 

(2914, (1937,))

### Create a where a row represents a vehicle and the columns are accel and decel

In [7]:
keep_vehicles = types_df.loc[(types_df['type_accel'] > 0) & (types_df['type_decel'] > 0)].index

In [8]:
both_vehicles = df.loc[df.vehicle_id.isin(keep_vehicles)]

In [9]:
groups = []
for name, group in both_vehicles[['vehicle_id', 'type', 'slope']].groupby(['vehicle_id']):
    # group[['type', 'slope']]
    accels = group.loc[group.type == 'accel', 'slope'].values
    decels = group.loc[group.type == 'decel', 'slope'].values
    # create combinations of accel and decel
    groups.extend(({'id': name, 'accel': accel, 'decel': decel}) for decel in decels for accel in accels)
    

In [10]:
accel_decel_df = pd.DataFrame(groups)

In [11]:
accel_decel_df.head()

,id,accel,decel
0,55583,0.786503,-1.706129
1,55587,1.674436,-0.988193
2,55587,1.674436,-1.784263
3,55596,1.383191,-1.004798
4,55596,1.383191,-1.669873


### Clean it up and get it ready for SUMO

In [12]:
accel_decel_df['decel'] = accel_decel_df['decel'].abs()
accel_decel_df.drop('id', axis=1, inplace=True)

### Add the Vehicle Type

In [13]:
accel_decel_df['vClass'] = 'passenger' #np.random.choice(['passenger', 'trailer'], N, p=[0.8, 0.2])
accel_decel_df['emissionClass'] = accel_decel_df['vClass'].map({
    'passenger': 'PHEMlight/PC_G_EU4',
    'trailer': 'PHEMlight/HDV_TT_D_EU6'
})

In [14]:
accel_decel_df.to_csv('rw_east_thru_summary_accel_decel.csv', index=False)